# LiDAR-Net train

## Create network

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import os
import pcl
import math
import yaml

import sys
sys.path.append("../src/")
from laserscan import LaserScan, SemLaserScan
from model import LiDAR_Model
from pointcloud_handling_jupyter import *

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
layers = 'xyzi' # ir xyz xyzi xyzir
yaml_path = '../config/semantic-kitti_GroundObject.yaml' #'../config/semantic-kitti_GroundObject.yaml' OR '../config/semantic-kitti.yaml'

In [3]:
# Generate color map and lookup tables

# Load configuration file
CFG = yaml.safe_load(open(yaml_path,'r'))

# Read kitti classes to color dictionary from configuration
KittiToColorDict = CFG['color_map']

# Read kitti to master project classes dictionary from configuration
KittiToProjectDict = CFG['learning_map']

# Read master project to kitti dictionary from configuration
ProjectToKittiDict = CFG['learning_map_inv']

# Create lookup table for kitti classes to color
maxkeyColor = max(KittiToColorDict.keys()) + 100 # +100 hack making lut bigger in case there are unknown labels
KittiToColor_LUT = np.zeros((maxkeyColor, 3), dtype=np.uint8)
KittiToColor_LUT[list(KittiToColorDict.keys())] = list(KittiToColorDict.values())

# Create lookup table for kitti classes to master project classes
maxkey = max(KittiToProjectDict.keys()) + 100 # +100 hack making lut bigger in case there are unknown labels 
maxvalue = max(KittiToProjectDict.values())
KittiToProject_LUT = np.zeros((maxkey), dtype=np.int32)
KittiToProject_LUT[list(KittiToProjectDict.keys())] = list(KittiToProjectDict.values())

# Create lookup table for master project classes to kitti classes
maxkeyInv = max(ProjectToKittiDict.keys()) + 100 # +100 hack making lut bigger in case there are unknown labels
ProjectToKitti_LUT = np.zeros((maxkeyInv), dtype=np.int32)
ProjectToKitti_LUT[list(ProjectToKittiDict.keys())] = list(ProjectToKittiDict.values())

In [4]:
model = LiDAR_Model(len(layers), CFG['num_classes'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 4)]   0         
_________________________________________________________________
res_context_block (ResContex (None, None, None, 32)    18912     
_________________________________________________________________
res_context_block_1 (ResCont (None, None, None, 32)    19808     
_________________________________________________________________
res_context_block_2 (ResCont (None, None, None, 32)    19808     
_________________________________________________________________
res_block (ResBlock)         ((None, None, None, 64),  87360     
_________________________________________________________________
res_block_1 (ResBlock)       ((None, None, None, 128), 346752    
_________________________________________________________________
res_block_2 (ResBlock)       ((None, None, None, 128), 428672

## Prepare data

In [5]:
BatchSize = 4 # für 64er pc => BatchSize=4 #für 16er pc & 5,xMill. Param => 32 
Epochs = 30 #TODO - mehr EPOCHS
LearningRate = 1e-5

In [6]:
def PointCloudGenerator(sample_paths, label_paths, batch_size, random=True): #add , pointcloud_size):
    """
    sample_paths = [sample_path1, sample_path2, ...]
    label_paths = [label_path1, label_path2, ...]
    """
    
    if random == True:
        sample_paths, label_paths = shuffle(sample_paths, label_paths, random_state=42)
        
    num_samples = len(sample_paths)

    while True: # Loop forever so the generator never terminates
        # Get index to start each batch: [0, batch_size, 2*batch_size, ..., max multiple of batch_size <= num_samples]
        for offset in range(0, num_samples, batch_size):
            # Get the samples + paths you'll use in this batch
            batch_sample_paths = sample_paths[offset:offset+batch_size]
            batch_label_paths = label_paths[offset:offset+batch_size]
            
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            y_train = []

            # For each example
            for batch_sample,_ in enumerate(batch_sample_paths):
                
                # Load points (X) and labels (y)
                PointCloud = SemLaserScan(20, KittiToColorDict, project=True, W=2048, H=64)
                
                current_sample_path = batch_sample_paths[batch_sample]
                current_label_path = batch_label_paths[batch_sample]
                
                current_sample = getSampleArrayFromPointCloud (PointCloud, current_sample_path, layers)
                current_sample = current_sample[0::2,0::4]
                
                current_label = getLabelArrayFromPointCloud(PointCloud, current_label_path, KittiToProject_LUT, maxvalue)
                current_label = current_label[0::2,0::4]
                
                # Add example to arrays
                X_train.append(current_sample)
                y_train.append(current_label)

            # Make sure they're numpy arrays (as opposed to lists)
            X_train = np.array(X_train)
            y_train = np.array(y_train)

            # The generator-y part: yield the next training batch            
            yield X_train, y_train

In [7]:
# Define labeled data path
PATH = '/data/kitti_data/dataset/sequences/'
# Get all labeled sequences
sequences = [PATH + i for i in sorted(os.listdir(PATH))]

sample_paths = []
label_paths = []

# foreach labeled sequence -> get sample and lable path
for i in sequences:
    sample = i + '/velodyne/'
    label = i + '/labels/'
    for s in sorted(os.listdir(sample)):
        sample_paths.append(sample + s)
    for s in sorted(os.listdir(label)):
        label_paths.append(label + s)

In [8]:
# Shuffle paths and split them into training & validation paths
train_sample_paths, val_sample_paths, train_label_paths, val_label_paths = train_test_split(sample_paths, label_paths, train_size=0.8, random_state=42)
val_sample_paths, eval_sample_paths, val_label_paths, eval_label_paths = train_test_split(val_sample_paths, val_label_paths, train_size=0.75, random_state=42)

# Create training and validation generators
datagen = PointCloudGenerator(train_sample_paths, train_label_paths, BatchSize)
val_gen = PointCloudGenerator(val_sample_paths, val_label_paths, BatchSize)
eval_gen = PointCloudGenerator(eval_sample_paths, eval_label_paths, BatchSize)

## Train network

In [9]:
Optimizer = Adam(learning_rate = LearningRate)
# Configure model for training
model.compile(
loss='categorical_crossentropy',
optimizer=Optimizer,
metrics=['accuracy']
)

In [10]:
TrainingHistory = model.fit(
    x=datagen,
    epochs=Epochs,
    verbose=1,
    validation_data=val_gen,
    steps_per_epoch = math.ceil(len(train_sample_paths)/BatchSize),
    validation_steps = math.ceil(len(val_sample_paths)/BatchSize)
)

Train for 4640 steps, validate for 870 steps
Epoch 1/30
4640/4640 [==============================] - 1355s 292ms/step - loss: 0.3223 - accuracy: 0.8811 - val_loss: 0.2081 - val_accuracy: 0.9279
Epoch 2/30
4640/4640 [==============================] - 1339s 289ms/step - loss: 0.1940 - accuracy: 0.9339 - val_loss: 0.1750 - val_accuracy: 0.9403
Epoch 3/30
4640/4640 [==============================] - 1339s 289ms/step - loss: 0.1714 - accuracy: 0.9420 - val_loss: 0.1599 - val_accuracy: 0.9459
Epoch 4/30
4640/4640 [==============================] - 1343s 289ms/step - loss: 0.1588 - accuracy: 0.9464 - val_loss: 0.1493 - val_accuracy: 0.9498
Epoch 5/30
4640/4640 [==============================] - 1343s 289ms/step - loss: 0.1497 - accuracy: 0.9494 - val_loss: 0.1413 - val_accuracy: 0.9523
Epoch 6/30
4640/4640 [==============================] - 1343s 289ms/step - loss: 0.1428 - accuracy: 0.9517 - val_loss: 0.1356 - val_accuracy: 0.9541
Epoch 7/30
4640/4640 [==============================] - 1344s

In [11]:
#model.save_weights('../weights/model_weights_IR_16_1024.h5')
#model.save_weights('../weights/model_weights_IR_64_2048.h5')
model.save_weights('../weights/model_weights_xyzi_16_1024_GroundDetection_Epoch30.h5')

## Evaluate network general

In [ ]:
#model.load_weights('../weights/model_weights_xyzi_16_1024.h5')
#model.load_weights('../weights/model_weights_IR_64_2048.h5')
Evaluation = model.evaluate(eval_gen, steps=math.ceil(len(eval_sample_paths)/BatchSize))
print(Evaluation)

# Debug - image creation
## Evaluate network visual

In [ ]:
# Load reduced model weights
model.load_weights('../weights/model_weights_xyzi_16_1024_GroundDetection_inklTerain.h5')

# Load full model weights
#model.load_weights('../weights/model_weights_IR_64_2048.h5')



In [ ]:
def PredictionToImage(Prediction):
    # Map masterproject classes to kitti classes 
    Prediction = ProjectToKitti_LUT[Prediction]
    # Map kitti classes to colors
    Image = KittiToColor_LUT[Prediction]
    Image = np.swapaxes(Image,0,1)
    Image = Image[...,[2,1,0]]
    return Image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Define path of test PoinCloud
TestPath = '/data/kitti_data/dataset/sequences'
CurrentTestPoinCloudPath = TestPath + '/04/velodyne/000008.bin'

# Create test PointCloud
TestPointCloud = SemLaserScan(20, KittiToColorDict, project=True, W=2048, H=64)

# Create full sample array from PoinCloud
CurrentTestSampleFull = getSampleArrayFromPointCloud (TestPointCloud, CurrentTestPoinCloudPath)
print(CurrentTestSampleFull.shape)
#print(np.min(CurrentTestSampleFull[:,:,2]))
CurrentTestSampleFull = np.expand_dims(CurrentTestSampleFull, axis=0)

# Create reduced sample array from full sample array
#print(CurrentTestSampleFull.shape)
CurrentTestSampleReduced = CurrentTestSampleFull[0::,0::2,0::4]
#print(np.min(CurrentTestSampleReduced[0,:,:,2]))
#print(CurrentTestSampleReduced.shape)

# Predict full PointCloud
PredictionFull = model.predict(CurrentTestSampleFull)
PredictionFull = np.argmax(PredictionFull,axis=3)
PredictionFull = PredictionFull.squeeze()

# Predict reduced PointCloud
PredictionReduced = model.predict(CurrentTestSampleReduced)
PredictionReduced = np.argmax(PredictionReduced,axis=3)
PredictionReduced = PredictionReduced.squeeze()

# Create images of predictions
PredictionFullImage = PredictionToImage(PredictionFull)
PredictionReducedImage = PredictionToImage(PredictionReduced)

# [optional] Spot special classes in image
#ids = np.where((CurrentTestSampleReduced[0,:,:,2] < -5))
#print(ids)
#PredictionReducedImage[ids] = [255, 0, 0]
#PredictionReducedImage = np.swapaxes(PredictionReducedImage,0,1)
#PredictionFullImage = np.swapaxes(PredictionFullImage,0,1)

# Show and save full prediction image
fig = plt.figure()
plt.imshow(PredictionFullImage)
plt.show
plt.imsave('../images/PredictionFull.png', PredictionFullImage)

# Show and save reduced prediction image
fig = plt.figure()
plt.imshow(PredictionReducedImage)
plt.show
plt.imsave('../images/PredictionReduced.png', PredictionReducedImage)

In [ ]:
CurrentTestLabelPath = TestPath + '/04/labels/000008.label'

CurrentLabelFull = getLabelArrayFromPointCloud (TestPointCloud, CurrentTestLabelPath)
CurrentLabelFull = np.expand_dims(CurrentLabelFull, axis=0)

CurrentLabelReduced = CurrentLabelFull[0::,0::2,0::4]

CurrentLabelFull = np.argmax(CurrentLabelFull,axis=3)
CurrentLabelFull = CurrentLabelFull.squeeze()

CurrentLabelReduced = np.argmax(CurrentLabelReduced,axis=3)
CurrentLabelReduced = CurrentLabelReduced.squeeze()

GroundTruthFullImage = PredictionToImage(CurrentLabelFull)
GroundTruthReducedImage = PredictionToImage(CurrentLabelReduced)

fig = plt.figure()
plt.imshow(GroundTruthFullImage)
plt.show
plt.imsave('../images/GroundTruthFullImage.png', GroundTruthFullImage)

fig = plt.figure()
plt.imshow(GroundTruthReducedImage)
plt.show
plt.imsave('../images/GroundTruthReducedImage.png', GroundTruthReducedImage)

In [ ]:
# Create intensity histogram of full kitti data
fig = plt.figure()
print(CurrentTestSampleFull.shape)
I = CurrentTestSampleFull[:,:,:,0].flatten()
# print(I.shape)
hist = plt.hist(I, bins='auto')
plt.title("Intensity histogram of full kitti data")
plt.show()

# Create intensity histogram of reduced kitti data
fig = plt.figure()
I = CurrentTestSampleReduced[:,:,:,0].flatten()
print(I.shape)
hist = plt.hist(I, bins='auto')  # arguments are passed to np.histogram
plt.title("Intensity histogram of reduced kitti data")
plt.show()

# Create range histogram of full kitti data
fig = plt.figure()
print(CurrentTestSampleFull.shape)
I = CurrentTestSampleFull[:,:,:,1].flatten()
# print(I.shape)
hist = plt.hist(I, bins='auto')
plt.title("Range histogram of full kitti data")
plt.show()

# Create range histogram of reduced kitti data
fig = plt.figure()
I = CurrentTestSampleReduced[:,:,:,1].flatten()
print(I.shape)
hist = plt.hist(I, bins='auto')  # arguments are passed to np.histogram
plt.title("Range histogram of reduced kitti data")
plt.show()

## Predict BugaLog data

In [ ]:
PCDPATH = '../data/pcd_files/'
current_pcd_path = PCDPATH + 'front_cloud_straße.pcd' #'top_cloud_git.pcd'

PointCloud = SemLaserScan(20, KittiToColorDict, project=True, W=1440, H=16, fov_up=15, fov_down=-15.0)

current_sample = getSampleArrayFromPointCloud_pcd(PointCloud, current_pcd_path, 1.13)
#print(current_sample.shape)
#current_sample[:,:,2] = current_sample[:,:,2] -0.4#1.13 #+0.7 #- 1.13
print(np.max(current_sample[:,:,2]))
print(np.min(current_sample[:,:,2]))

#current_sample = current_pcd
current_sample = np.expand_dims(current_sample, axis=0)

#current_sample = current_sample[0::,0::2,0::4]
#current_sample[np.where(current_sample[:,:,:,0] > 0.4)] = [-1, -1]

Prediction = model.predict(current_sample)

Prediction = np.argmax(Prediction,axis=3)

Prediction = Prediction.squeeze()

BugaLogImage = PredictionToImage(Prediction)

plt.imshow(BugaLogImage)
plt.show
plt.imsave('../images/BugaLogImage.png', BugaLogImage)

In [ ]:
fig = plt.figure()
I = current_sample[:,:,:,0].flatten()
print(I.shape)
hist = plt.hist(I, bins='auto')  # arguments are passed to np.histogram
plt.title("Intensity histogram of buga log data")
plt.show()

fig = plt.figure()
R = current_sample[:,:,:,1].flatten()
print(R.shape)
hist = plt.hist(R, bins='auto')  # arguments are passed to np.histogram
plt.title("Range histogram of buga log data")
plt.show()